In [4]:
import yfinance as yf
from datetime import datetime
import warnings

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")


## Yahoo Fundamental

In [5]:
def get_yahoo_economic_calendar(table_class: str = None) -> pd.DataFrame:
    """
    Extrat data from trading economics callendar.

    Parameters:
        classe_tabela (str): Class CSS from table to scrap (optional).

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get("https://tradingeconomics.com/calendar", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        if table_class:
            tabela = soup.find('table', {'class': table_class})
        else:
            tabela = soup.find('table')
        
        headers = [th.text.strip() for th in tabela.find_all('th')]
        print(headers)
        rows = []
        for row in tabela.find_all('tr')[1:]:
            cols = [td.text.strip() for td in row.find_all('td')]
            if cols: 
                rows.append(cols)

        df = pd.DataFrame(rows, columns=headers if headers else None)
        

    except requests.exceptions.RequestException as e:
        print(f"Error accessing URL: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

    return df


In [6]:
def get_whalewisdom(ticker: str) -> pd.DataFrame:
    """
    Extracts data from WhaleWisdom for a given stock ticker.

    Parameters:
        ticker (str): Stock ticker symbol.

    Returns:
        pd.DataFrame: DataFrame with table content.
    """

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(f"https://whalewisdom.com/stock/{ticker}", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Encontrar a div com a classe "v-data-table__wrapper"
        div_wrapper = soup.find('div', {'class': 'v-window__container'})
       
        if not div_wrapper:
            raise ValueError("Div com a classe 'v-window__container' não encontrada.")

        # Buscar a primeira tabela dentro desta div
        tabela = div_wrapper.find('table')
        if not tabela:
            raise ValueError("Nenhuma tabela encontrada dentro da div especificada.")
        
        # Coletar os cabeçalhos da tabela ignorando ícones
        headers = [th.find('span').text.strip() for th in tabela.find_all('th') if th.find('span')]
        # print(headers)

        # Encontrando todas as linhas dentro da tabela (tbody > tr)
        rows = tabela.find("tbody").find_all("tr")
        print(rows)
        # Extraindo os dados de cada linha
        data = []
        for row in rows:
            columns = row.find_all("td")

            # Pegando os valores correspondentes
            institution_name = columns[0].text.strip()
            shares_held = columns[2].text.strip()
            portfolio_value = columns[3].text.strip()
            percentage_ownership = columns[4].text.strip()
            last_report_date = columns[-1].text.strip()

            data.append({
                "Instituição": institution_name,
                "Ações Detidas": shares_held,
                "Valor do Portfólio": portfolio_value,
                "Percentual de Participação": percentage_ownership,
                "Última Atualização": last_report_date
            })
        #print(data)
    
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a URL: {e}")
    except ValueError as e:
        print(f"Erro na extração de dados: {e}")
    except Exception as e:
        print(f"Erro inesperado: {e}")
    
    return pd.DataFrame()

In [7]:
df_whale_s = get_whalewisdom(ticker = "ctre")
df_whale_s

NameError: name 'requests' is not defined

In [ ]:
base_url = 'https://whalewisdom.com/shell/command.html?args=%7B%22command%22:%22filer_lookup%22,%20%22name%22:%22berkshire%22%7D'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "application/json"
}

response = requests.get(base_url, headers=headers)
response.raise_for_status()

response.text

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <title>Track Hedge Funds Using 13F Filings </title>\n    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, minimal-ui">\n            <meta name="description" content="WhaleWisdom tracks 13F,  Schedule 13D, and 13G EDGAR filings by hedge funds. Hedge Fund Whale Backtesting and search tools "/>\n    <link rel="shortcut icon" type="image/x-icon" href="https://d27mjrcvcy56qq.cloudfront.net/images/favicon.ico"/>\n    <link rel="apple-touch-icon" sizes="57x57" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-57x57.png">\n    <link rel="apple-touch-icon" sizes="60x60" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-60x60.png">\n    <link rel="apple-touch-icon" sizes="72x72" href="https://d27mjrcvcy56qq.cloudfront.net/images/apple-touch-icon-72x72.png">\n    <link rel="apple-touch-icon" sizes="76x76" href="https://d27mjrcvcy56

In [ ]:
import hashlib
import hmac
import base64
import json
import time
import urllib.parse
import requests

class WhaleWisdom:
    def __init__(self, secret_key, shared_key):
        self.secret_key = secret_key
        self.shared_key = shared_key
        self.timestamp = self.get_timestamp()

    def get_timestamp(self):
        """Gera o timestamp no formato UTC"""
        return time.strftime('%Y-%m-%dT%H:%M:%SZ', time.gmtime())

    def signature(self, args):
        """Gera a assinatura HMAC-SHA1 em Base64"""
        message = f"{args}\n{self.timestamp}".encode('utf-8')
        hmac_digest = hmac.new(self.secret_key.encode('utf-8'), message, hashlib.sha1).digest()
        return base64.b64encode(hmac_digest).decode('utf-8')

    def encode(self, string):
        """Codifica a string para URL"""
        return urllib.parse.quote(string, safe='')

    def endpoint(self, args):
        """Constrói a URL da API do WhaleWisdom"""
        encoded_args = self.encode(json.dumps(args))
        api_sig = self.signature(json.dumps(args))
        encoded_timestamp = self.encode(self.timestamp)

        return f"https://whalewisdom.com/shell/command.json?args={encoded_args}&api_shared_key={self.shared_key}&api_sig={api_sig}&timestamp={encoded_timestamp}"

    def request(self, args):
        """Faz a requisição à API"""
        url = self.endpoint(args)
        response = requests.get(url)

        if response.status_code == 200:
            return response.text
        else:
            print(f"Erro {response.status_code}: {response.text}")
            return None


# **Exemplo de Uso**
WW_SHARED_KEY = "aTWxWrcILEOGqkAPZtAL"
WW_SECRET_KEY = "JGlAleUrGy1bWaLoiWwESNIbTHaovA0ocKOnr4MW"

ww = WhaleWisdom(secret_key=WW_SECRET_KEY, shared_key=WW_SHARED_KEY)

# Obter as participações (holdings) de um gestor específico
response = ww.request({
    "command": "stock_lookup",
    "symbol": "ctre"
})

data = json.loads(response)

stock_id = data["stocks"][0]["id"]
name = data["stocks"][0]["name"]
status = data["stocks"][0]["status"]
link = data["stocks"][0]["link"]

stock_id

167941

In [ ]:
response = ww.request(
{"command":"holdings","stock_ids":[167941],"filer_ids":[373], "limit": 10, "all_quarters":0})
response


'{"errors":["Subscription limit has been reached for current period"]}'

In [ ]:
response

'{"errors":["Subscription limit has been reached for current period"]}'

## STOCKS



In [44]:
import yfinance as yf
import pandas as pd
import talib
from django.http import JsonResponse
from datetime import datetime, timedelta

# S&P 500 → ^GSPC
# Dow Jones → ^DJI
# Nasdaq 100 → ^NDX
# Russell 2000 → ^RUT
# DAX (Alemanha) → ^GDAXI
# Definir o ticker da ação
symbol = "AAPL"  # Apple Inc.
# symbol = "^GSPC"  # Apple Inc.
# symbol = "XLK"  # Apple Inc.

# Definir período e intervalo
period = "1y"       # Período de 1 mês
interval = "1d"      # Dados diários

yahoo_symbol_info = yf.Ticker(symbol).info

In [45]:
def get_sector_etf_info(sector_name, search_value: str = "info" ):
    """
    Return information about symbol ETF sector.
    """
    sector_map = {
        "Technology": "XLK",
        "Financial Services": "XLF",
        "Consumer Cyclical": "XLY",
        "Healthcare": "XLV",
        "Communication Services": "XLC",
        "Industrials": "XLI",
        "Consumer Defensive": "XLP",
        "Energy": "XLE",
        "Real Estate": "XLRE",
        "Basic Materials": "XLB",
        "Utilities": "XLU",
    }

    etf_symbol = sector_map.get(sector_name)
    if not etf_symbol:
        return "N/A"

    ticker = yf.Ticker(etf_symbol)

    if search_value == "info":
        return ticker.info
    else:
        return ticker.info.get(search_value, "N/A")
        

# Exemplo de uso
sector = "Utilities"
info = get_sector_etf_info(sector, "info")
print(f"Informações do setor {sector}:")
print(info)

Informações do setor Utilities:
{'longBusinessSummary': 'In seeking to track the performance of the index, the fund employs a replication strategy. It generally invests substantially all, but at least 95%, of its total assets in the securities comprising the index. The index includes securities of companies from the following industries: electric utilities; water utilities; multi-utilities; independent power and renewable electricity producers; and gas utilities. The fund is non-diversified.', 'companyOfficers': [], 'executiveTeam': [], 'maxAge': 86400, 'priceHint': 2, 'previousClose': 78.94, 'open': 78.95, 'dayLow': 78.78, 'dayHigh': 79.37, 'regularMarketPreviousClose': 78.94, 'regularMarketOpen': 78.95, 'regularMarketDayLow': 78.78, 'regularMarketDayHigh': 79.37, 'trailingPE': 20.540232, 'volume': 8183031, 'regularMarketVolume': 8183031, 'averageVolume': 10014246, 'averageVolume10days': 11294390, 'averageDailyVolume10Day': 11294390, 'bid': 78.75, 'ask': 79.38, 'bidSize': 12, 'askSize

In [3]:
sector_info = yf.Ticker("T").info
sector_info

{'address1': '208 South Akard Street',
 'city': 'Dallas',
 'state': 'TX',
 'zip': '75202',
 'country': 'United States',
 'phone': '210 821 4105',
 'website': 'https://www.att.com',
 'industry': 'Telecom Services',
 'industryKey': 'telecom-services',
 'industryDisp': 'Telecom Services',
 'sector': 'Communication Services',
 'sectorKey': 'communication-services',
 'sectorDisp': 'Communication Services',
 'longBusinessSummary': 'AT&T Inc. provides telecommunications and technology services worldwide. The company operates through two segments, Communications and Latin America. The Communications segment offers wireless voice and data communications services; and sells handsets, wireless data cards, wireless computing devices, carrying cases/protective covers, and wireless chargers through its own company-owned stores, agents, and third-party retail stores. It also provides AT&T Dedicated Internet, fiber ethernet and broadband, fixed wireless, and hosted and managed professional services; a

In [48]:
yahoo_symbol_income = yf.Ticker(symbol).income_stmt
yahoo_symbol_income.index

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBITDA',
       'EBIT', 'Net Interest Income', 'Interest Expense', 'Interest Income',
       'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Total Operating Income As Reported',
       'Diluted Average Shares', 'Basic Average Shares', 'Diluted EPS',
       'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Other Non Operating Income Expenses',
       'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Interest Income Non Operating',
       'Operating Inco

In [57]:
net_income = yahoo_symbol_income.loc['Net Income']
net_income

2024-09-30    93736000000.0
2023-09-30    96995000000.0
2022-09-30    99803000000.0
2021-09-30    94680000000.0
2020-09-30              NaN
Name: Net Income, dtype: object

In [58]:
total_revenue = yahoo_symbol_income.loc['Total Revenue']
total_revenue

2024-09-30    391035000000.0
2023-09-30    383285000000.0
2022-09-30    394328000000.0
2021-09-30    365817000000.0
2020-09-30               NaN
Name: Total Revenue, dtype: object

In [ ]:
net_income = yahoo_symbol_income.loc['Net Income'].iloc[0]
net_income

total_revenue = yahoo_symbol_income.loc['Total Revenue'].iloc[0]
cost_of_revenue = yahoo_symbol_income.loc['Cost Of Revenue'].iloc[0]
gross_profit = yahoo_symbol_income.loc['Gross Profit'].iloc[0]



In [74]:
try:
    yahoo_symbol_income = yf.Ticker(symbol).income_stmt
except:
    yahoo_symbol_income = pd.DataFrame()

# Inicializar todas as variáveis com "N/A"
net_income = "N/A"
total_revenue = "N/A"
cost_of_revenue = "N/A"
gross_profit = "N/A"

# net_income
if 'Net Income' in yahoo_symbol_income.index:
    net_income = yahoo_symbol_income.loc['Net Income']
    if pd.isna(net_income).all():
        net_income = "N/A"

# total_revenue
if 'Total Revenue' in yahoo_symbol_income.index:
    total_revenue = yahoo_symbol_income.loc['Total Revenue']
    if pd.isna(total_revenue).all():
        total_revenue = "N/A"

# cost_of_revenue
if 'Cost Of Revenue' in yahoo_symbol_income.index:
    cost_of_revenue = yahoo_symbol_income.loc['Cost Of Revenue']
    if pd.isna(cost_of_revenue).all():
        cost_of_revenue = "N/A"

# gross_profit
if 'Gross Profit' in yahoo_symbol_income.index:
    gross_profit = yahoo_symbol_income.loc['Gross Profit']
    if pd.isna(gross_profit).all():
        gross_profit = "N/A"

In [92]:
def get_yoy_metric(series):
    if isinstance(series, pd.Series):
        yoy_growth = ((series - series.shift(-1)) / series.shift(-1)) * 100
        yoy_growth = yoy_growth.dropna()
        
    return yoy_growth

def get_cagr_metric(series):
    series = series.dropna()
    if len(series) >= 2:
        start_value = series.iloc[-1]
        end_value = series.iloc[0]
        num_years = len(series) - 1
        cagr = ((end_value / start_value) ** (1 / num_years)) - 1

    cagr_percent = cagr * 100 if cagr != "N/A" else "N/A"

    return cagr_percent

In [93]:
get_yoy_metric(cost_of_revenue)

2024-09-30    -1.76756
2023-09-30   -4.208977
2022-09-30    4.960536
Name: Cost Of Revenue, dtype: object

In [95]:
get_yoy_metric(total_revenue)

2024-09-30    2.021994
2023-09-30   -2.800461
2022-09-30    7.793788
Name: Total Revenue, dtype: object

In [ ]:
get_cagr_metric(cost_of_revenue)

-0.4131655096255016

In [96]:
get_cagr_metric(total_revenue)

2.247001899501888

In [5]:
def get_symbol_fundamental_info(symbol : str):
    '''
    Return detailed fundamental information about asset
    '''
    try:
        yahoo_symbol_info = yf.Ticker(symbol).info
    except:
        yahoo_symbol_info = {}
    try:
        yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
    except:
        yahoo_symbol_balancesheet = pd.DataFrame()
    try:
        yahoo_symbol_income = yf.Ticker(symbol).income_stmt
    except:
        yahoo_symbol_income = pd.DataFrame()
    
    # Inicializar todas as variáveis com "N/A"
    total_equity = "N/A"
    cash_equivalents_short_term_investments = "N/A"
    cash_and_cash_equivalents = "N/A"
    current_liabilities = "N/A"
    cash_ratio = "N/A"
    ebit = "N/A"
    interest_expenses = "N/A"
    interest_coverage_ratio = "N/A"
    total_assets = "N/A"
    debt_to_assets_ratio = "N/A"

    # total_equity
    if 'StockholdersEquity' in yahoo_symbol_balancesheet.index:
        total_equity = yahoo_symbol_balancesheet.loc['StockholdersEquity'].iloc[0]
        if pd.isna(total_equity) or total_equity == 0:
            total_equity = "N/A"
    else:
        total_equity = "N/A"
        
    # cash_equivalents_short_term_investments
    if 'CashCashEquivalentsAndShortTermInvestments' in yahoo_symbol_balancesheet.index:
        cash_equivalents_short_term_investments = yahoo_symbol_balancesheet.loc['CashCashEquivalentsAndShortTermInvestments'].iloc[0]
        if pd.isna(cash_equivalents_short_term_investments) or cash_equivalents_short_term_investments == 0:
            cash_equivalents_short_term_investments = "N/A"
    else:
        cash_equivalents_short_term_investments = "N/A"

    # cash_and_cash_equivalents
    if 'CashAndCashEquivalents' in yahoo_symbol_balancesheet.index:
        cash_and_cash_equivalents = yahoo_symbol_balancesheet.loc['CashAndCashEquivalents'].iloc[0]
        if pd.isna(cash_and_cash_equivalents) or cash_and_cash_equivalents == 0:
            cash_and_cash_equivalents = "N/A"
    else:
        cash_and_cash_equivalents = "N/A"

    # current_liabilities
    if 'CurrentLiabilities' in yahoo_symbol_balancesheet.index:
        current_liabilities = yahoo_symbol_balancesheet.loc['CurrentLiabilities'].iloc[0]
        if pd.isna(current_liabilities) or current_liabilities == 0:
            current_liabilities = "N/A"
    else:
        current_liabilities = "N/A"

    # cash_ratio
    if cash_and_cash_equivalents != "N/A" and current_liabilities != "N/A" and current_liabilities != 0:
        cash_ratio = cash_and_cash_equivalents / current_liabilities
    else:
        cash_ratio = "N/A"

    # ebit
    if 'EBIT' in yahoo_symbol_income.index:
        ebit = yahoo_symbol_income.loc["EBIT"].iloc[0]
        if pd.isna(ebit) or ebit == 0:
            ebit = "N/A"

    # interest_expenses
    if 'InterestExpense' in yahoo_symbol_income.index:
        interest_expenses = yahoo_symbol_income.loc["InterestExpense"].iloc[0]
        if pd.isna(interest_expenses) or interest_expenses == 0:
            interest_expenses = "N/A"

    # interest_coverage_ratio
    if ebit != "N/A" and interest_expenses not in ["N/A", 0]:
        interest_coverage_ratio = ebit / interest_expenses
    else:
        interest_coverage_ratio = "N/A"

    # total_assets
    if 'TotalAssets' in yahoo_symbol_balancesheet.index:
        total_assets = yahoo_symbol_balancesheet.loc['TotalAssets'].iloc[0]
        if pd.isna(total_assets) or total_assets == 0:
            total_assets = "N/A"
    else:
        total_assets = "N/A"

    # debt_to_assets_ratio 
    total_debt = yahoo_symbol_info.get("totalDebt", "N/A")
    if total_assets != "N/A" and total_debt != "N/A":
        debt_to_assets_ratio = total_debt / total_assets if total_assets != 0 else "N/A"
    else:
        debt_to_assets_ratio = "N/A"

    # PE adjusted to market
    sector = yahoo_symbol_info.get("sector")
    sector_pe = get_sector_etf_info(sector, "trailingPE")
    
    yahoo_symbol_info = yf.Ticker(symbol).info
    yahoo_symbol_fundamental_info = {
        "liquidity_and_solvency": {
            # Liquidez e Solvência: Capacidade de pagamento
            "QuickRatio": yahoo_symbol_info.get("quickRatio", "N/A"),
            "CurrentRatio": yahoo_symbol_info.get("currentRatio", "N/A"),
            "TotalCash": yahoo_symbol_info.get("totalCash", "N/A"),
            "TotalDebt": yahoo_symbol_info.get("totalDebt", "N/A"),
            "TotalEquity": total_equity,
            # Aux
            "CashCashEquivalentsAndShortTermInvestments": cash_equivalents_short_term_investments,
            "CashAndCashEquivalents": cash_and_cash_equivalents,
            "CurrentLiabilities": current_liabilities,
            "EBIT": ebit,
            "InteresExpenses": interest_expenses,
            # Curto Prazo
            "CashRatio": cash_ratio,
            "OperatingCashFlow": yahoo_symbol_info.get("operatingCashflow", "N/A"),
            # Longo Prazo
            "DebttoEquity": yahoo_symbol_info.get("debtToEquity", "N/A"),
            "InterestCoverageRatio": interest_coverage_ratio,
            "DebttoAssetsRatio": debt_to_assets_ratio,
        },
        "profitability": {
            "grossMargins": yahoo_symbol_info.get("grossMargins", "N/A"),
            "operatingMargins": yahoo_symbol_info.get("operatingMargins", "N/A"),
            "EBITDAMargins": yahoo_symbol_info.get("ebitdaMargins", "N/A"),
            "NetIncome": yahoo_symbol_info.get("netIncomeToCommon", "N/A"),
            "ProfitMargin": yahoo_symbol_info.get("profitMargins", "N/A"),
            "returnOnAssetsROA": yahoo_symbol_info.get("returnOnAssets", "N/A"),
            "returnOnEquityROE": yahoo_symbol_info.get("returnOnEquity", "N/A"),
        },
        "growth": {
            "revenueGrowth": yahoo_symbol_info.get("revenueGrowth", "N/A"),
            "earningsQuarterlyGrowth": yahoo_symbol_info.get("earningsQuarterlyGrowth", "N/A"),
            "earningsGrowth": yahoo_symbol_info.get("earningsGrowth", "N/A"),
        },
        "valuation": {
            "trailingPE": yahoo_symbol_info.get("trailingPE", "N/A"),
            "sectorTrailingPE": sector_pe,
            "forwardPE": yahoo_symbol_info.get("forwardPE", "N/A"),
            "PEGRatio": yahoo_symbol_info.get("trailingPegRatio", "N/A"),
            "PBRatio": yahoo_symbol_info.get("priceToBook", "N/A"),
            "enterpriseToEbitda": yahoo_symbol_info.get("enterpriseToEbitda", "N/A"),
        },
        "dividends_and_buybacks": {
            "dividendYield": yahoo_symbol_info.get("dividendYield", "N/A"),
            "payoutRatio": yahoo_symbol_info.get("payoutRatio", "N/A"),
            "fiveYearAvgDividendYield": yahoo_symbol_info.get("fiveYearAvgDividendYield", "N/A"),
        },
        "market_risk_and_sentiment": {
            "beta": yahoo_symbol_info.get("beta", "N/A"),
            "auditRisk": yahoo_symbol_info.get("auditRisk", "N/A"),
            "boardRisk": yahoo_symbol_info.get("boardRisk", "N/A"),
            "sharesPercentSharesOut": yahoo_symbol_info.get("sharesPercentSharesOut", "N/A"),
            "recommendationMean": yahoo_symbol_info.get("recommendationMean", "N/A"),
            "targetMeanPrice": yahoo_symbol_info.get("targetMeanPrice", "N/A")
        }
    }
    return yahoo_symbol_fundamental_info

In [6]:
metrics = get_symbol_fundamental_info("AMD")

In [7]:
metrics

{'liquidity_and_solvency': {'QuickRatio': 1.657,
  'CurrentRatio': 2.616,
  'TotalCash': 5132000256,
  'TotalDebt': 2320999936,
  'TotalEquity': 'N/A',
  'CashCashEquivalentsAndShortTermInvestments': 'N/A',
  'CashAndCashEquivalents': 'N/A',
  'CurrentLiabilities': 'N/A',
  'EBIT': 2081000000.0,
  'InteresExpenses': 'N/A',
  'CashRatio': 'N/A',
  'OperatingCashFlow': 3040999936,
  'DebttoEquity': 4.032,
  'InterestCoverageRatio': 'N/A',
  'DebttoAssetsRatio': 'N/A'},
 'profitability': {'grossMargins': 0.53019,
  'operatingMargins': 0.13803001,
  'EBITDAMargins': 0.19973,
  'NetIncome': 1640999936,
  'ProfitMargin': 0.06364,
  'returnOnAssetsROA': 0.019019999,
  'returnOnEquityROE': 0.02893},
 'growth': {'revenueGrowth': 0.242,
  'earningsQuarterlyGrowth': -0.277,
  'earningsGrowth': -0.297},
 'valuation': {'trailingPE': 104.59,
  'sectorTrailingPE': 34.21057,
  'forwardPE': 20.507843,
  'PEGRatio': 0.405,
  'PBRatio': 2.946861,
  'enterpriseToEbitda': 31.225},
 'dividends_and_buybacks'

In [24]:
if 'QuickRatio' in metrics.get('liquidity_and_solvency', {}):
    quick_ratio = metrics.get('liquidity_and_solvency', {}).get('QuickRatio', 'N/A')
    print(f"Quick Ratio: {quick_ratio}")
else:
    print("No Ratio")


Quick Ratio: 0.783


In [21]:
a = metrics.get('liquidity_and_solvency', {}).get('QuickRatio', 'N/A')
a

0.783

In [16]:
def evaluate_metrics(metrics):
    """
    Evaluate fundamental metrics.
    """

    evaluated_metrics = {}

    if 'QuickRatio' in metrics.get('liquidity_and_solvency', {}):
        quick_ratio = metrics.get('liquidity_and_solvency', {}).get('QuickRatio', 'N/A')
        if quick_ratio == "N/A":
            evaluated_metrics["QuickRatio"] = "N/A"
        elif quick_ratio < 0.75:
            evaluated_metrics["QuickRatio"] = "Very Negative"
        elif quick_ratio < 1:
            evaluated_metrics["QuickRatio"] = "Negative"
        elif quick_ratio < 1.5:
            evaluated_metrics["QuickRatio"] = "Neutral"
        elif quick_ratio < 2:
            evaluated_metrics["QuickRatio"] = "Positive"
        else:
            evaluated_metrics["QuickRatio"] = "Very Positive"

    if 'CurrentRatio' in metrics.get('liquidity_and_solvency', {}):
        current_ratio = metrics.get('liquidity_and_solvency', {}).get('CurrentRatio', 'N/A')
        if current_ratio == "N/A":
            evaluated_metrics["CurrentRatio"] = "N/A"
        elif current_ratio < 0.75:
            evaluated_metrics["CurrentRatio"] = "Very Negative"
        elif current_ratio < 1:
            evaluated_metrics["CurrentRatio"] = "Negative"
        elif current_ratio < 1.5:
            evaluated_metrics["CurrentRatio"] = "Neutral"
        elif current_ratio < 2:
            evaluated_metrics["CurrentRatio"] = "Positive"
        else:
            evaluated_metrics["CurrentRatio"] = "Very Positive"

    if 'CashRatio' in metrics.get('liquidity_and_solvency', {}):
        cash_ratio = metrics.get('liquidity_and_solvency', {}).get('CashRatio', 'N/A')
        if cash_ratio == "N/A":
            evaluated_metrics["CashRatio"] = "N/A"
        elif cash_ratio < 0.75:
            evaluated_metrics["CashRatio"] = "Very Negative"
        elif cash_ratio < 1:
            evaluated_metrics["CashRatio"] = "Negative"
        elif cash_ratio < 1.5:
            evaluated_metrics["CashRatio"] = "Neutral"
        elif cash_ratio < 2:
            evaluated_metrics["CashRatio"] = "Positive"
        else:
            evaluated_metrics["CashRatio"] = "Very Positive"

    if 'DebttoEquity' in metrics.get('liquidity_and_solvency', {}):
        debt_to_equity = metrics.get('liquidity_and_solvency', {}).get('DebttoEquity', 'N/A')
        if debt_to_equity == "N/A":
            evaluated_metrics["DebttoEquity"] = "N/A"
        elif debt_to_equity > 2.5:
            evaluated_metrics["DebttoEquity"] = "Very Negative"
        elif debt_to_equity > 1.8:
            evaluated_metrics["DebttoEquity"] = "Negative"
        elif debt_to_equity > 1.2:
            evaluated_metrics["DebttoEquity"] = "Neutral"
        elif debt_to_equity > 0.7:
            evaluated_metrics["DebttoEquity"] = "Positive"
        else:
            evaluated_metrics["DebttoEquity"] = "Very Positive"

    if 'DebttoAssetsRatio' in metrics.get('liquidity_and_solvency', {}):
        debt_to_assets = metrics.get('liquidity_and_solvency', {}).get('DebttoAssetsRatio', 'N/A')
        if debt_to_assets == "N/A":
            evaluated_metrics["DebttoAssetsRatio"] = "N/A"
        elif debt_to_assets > 2.5:
            evaluated_metrics["DebttoAssetsRatio"] = "Very Negative"
        elif debt_to_assets > 1.8:
            evaluated_metrics["DebttoAssetsRatio"] = "Negative"
        elif debt_to_assets > 1.2:
            evaluated_metrics["DebttoAssetsRatio"] = "Neutral"
        elif debt_to_assets > 0.7:
            evaluated_metrics["DebttoAssetsRatio"] = "Positive"
        else:
            evaluated_metrics["DebttoAssetsRatio"] = "Very Positive"

    if 'InterestCoverageRatio' in metrics.get('liquidity_and_solvency', {}):
        interest_cover_ratio = metrics.get('liquidity_and_solvency', {}).get('InterestCoverageRatio', 'N/A')
        if interest_cover_ratio == "N/A":
            evaluated_metrics["InterestCoverageRatio"] = "N/A"
        elif interest_cover_ratio < 1:
            evaluated_metrics["InterestCoverageRatio"] = "Very Negative"
        elif interest_cover_ratio < 2:
            evaluated_metrics["InterestCoverageRatio"] = "Negative"
        elif interest_cover_ratio < 3:
            evaluated_metrics["InterestCoverageRatio"] = "Neutral"
        elif interest_cover_ratio < 5:
            evaluated_metrics["InterestCoverageRatio"] = "Positive"
        else:
            evaluated_metrics["InterestCoverageRatio"] = "Very Positive"

    if "trailingPE" in metrics.get('valuation', {}) and \
    "sectorTrailingPE" in metrics.get('valuation', {}) and \
    "forwardPE" in metrics.get('valuation', {}):
        try:
            trailing_pe = metrics['valuation'].get("trailingPE", "N/A")
            sector_pe = metrics['valuation'].get("sectorTrailingPE", "N/A")
            forward_pe = metrics['valuation'].get("forwardPE", "N/A")
            
            if trailing_pe == "N/A" or sector_pe == "N/A" or forward_pe == "N/A":
                evaluated_metrics["trailingPE"] = "N/A"
            else:
                trailing_pe = float(trailing_pe)
                sector_pe = float(sector_pe)
                forward_pe = float(forward_pe)

                score_pe = 0

                if trailing_pe < sector_pe:
                    score_pe += 1
                elif trailing_pe > sector_pe:
                    score_pe -= 1

                if forward_pe < sector_pe:
                    score_pe += 1
                elif forward_pe > sector_pe:
                    score_pe -= 1

                if trailing_pe > forward_pe:
                    score_pe += 1
                elif trailing_pe < forward_pe:
                    score_pe -= 1

                if score_pe >= 3:
                    evaluated_metrics["trailingPE"] = "Very Low Undervalued"
                elif score_pe == 2:
                    evaluated_metrics["trailingPE"] = "Low Undervalued"
                elif score_pe == 1:
                    evaluated_metrics["trailingPE"] = "Undervalued"
                elif score_pe == 0:
                    evaluated_metrics["trailingPE"] = "Neutral Valued"
                elif score_pe == -1:
                    evaluated_metrics["trailingPE"] = "Overvalued"
                elif score_pe == -2:
                    evaluated_metrics["trailingPE"] = "High Overvalued"
                else:
                    evaluated_metrics["trailingPE"] = "Very High Overvalued"

        except (ValueError, TypeError):
            evaluated_metrics["trailingPE"] = "N/A"
            
    return evaluated_metrics if evaluated_metrics else "Indefinido"


In [17]:
evaluation_metrics = evaluate_metrics(metrics)

In [ ]:
evaluation_metrics

{'QuickRatio': 'Positive',
 'CurrentRatio': 'Very Positive',
 'CashRatio': 'N/A',
 'DebttoEquity': 'Very Negative',
 'DebttoAssetsRatio': 'N/A',
 'InterestCoverageRatio': 'N/A',
 'trailingPE': 'Undervalued'}

In [1]:
yahoo_symbol_info = yf.Ticker("T").info
yahoo_symbol_balancesheet = yf.Ticker(symbol).balance_sheet
yahoo_symbol_income = yf.Ticker(symbol).income_stmt

NameError: name 'yf' is not defined

In [4]:
yahoo_symbol_income.loc["EBIT"].iloc[0]

123216000000.0

In [ ]:
# Market Price vs Value 

In [7]:
yahoo_symbol_info

{'longBusinessSummary': 'In seeking to track the performance of the index, the fund employs a replication strategy, which means that the fund typically invests in substantially all of the securities represented in the index in approximately the same proportions as the index. It generally invests substantially all, but at least 95%, of its total assets in the securities comprising the index. The fund is non-diversified.',
 'companyOfficers': [],
 'executiveTeam': [],
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 211.68,
 'open': 211.07,
 'dayLow': 206.8475,
 'dayHigh': 211.54,
 'regularMarketPreviousClose': 211.68,
 'regularMarketOpen': 211.07,
 'regularMarketDayLow': 206.8475,
 'regularMarketDayHigh': 211.54,
 'trailingPE': 32.97669,
 'volume': 6183607,
 'regularMarketVolume': 6183607,
 'averageVolume': 4943936,
 'averageVolume10days': 7075400,
 'averageDailyVolume10Day': 7075400,
 'bid': 207.8,
 'ask': 208.79,
 'bidSize': 8,
 'askSize': 8,
 'yield': 0.0068,
 'totalAssets': 7091

In [9]:
# yahoo_symbol_cashflow = yf.Ticker(symbol).cash_flow
# yahoo_symbol_cashflow.index

In [10]:
# revenue_history = yahoo_symbol_info.get("netIncomeToCommon")
# revenue_history

In [11]:
# https://medium.com/towards-data-science/pivot-points-calculation-in-python-for-day-trading-659c1e92d323


# ### ZONAS PIVOT! ###

## Data Collection Binance


In [ ]:
import requests

url = "https://api.binance.com/api/v3/ticker/price"
params = {"symbol": "BTCUSDT"}

response = requests.get(url, params=params)

if response.status_code == 200:
    symbol = response.json()["symbol"]
    price = response.json()["price"]
else:
    error = response.status_code

In [ ]:
from decimal import Decimal
from datetime import datetime


def get_crypto_symbol_24h(symbol : str):

    url = "https://api.binance.com/api/v3/ticker/24hr"
    params = {"symbol": symbol}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        priceChangePercent = Decimal(response.json()["priceChangePercent"])
        weightedAvgPrice = Decimal(response.json()["weightedAvgPrice"])
        prevClosePrice = Decimal(response.json()["prevClosePrice"])
        priceChange = Decimal(response.json()["priceChange"])
        lastPrice = Decimal(response.json()["lastPrice"])
        lastQty = Decimal(response.json()["lastQty"])
        bidPrice = Decimal(response.json()["bidPrice"])
        bidQty = Decimal(response.json()["bidQty"])
        askPrice = Decimal(response.json()["askPrice"])
        askQty = Decimal(response.json()["askQty"])
        openPrice = Decimal(response.json()["openPrice"])
        highPrice = Decimal(response.json()["highPrice"])
        lowPrice = Decimal(response.json()["lowPrice"])
        volume = Decimal(response.json()["volume"])
        quoteVolume = Decimal(response.json()["quoteVolume"])
        openTime = datetime.fromtimestamp(response.json()["openTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
        closeTime = datetime.fromtimestamp(response.json()["closeTime"] / 1000).strftime("%d-%m-%Y %H:%M:%S")
        firstId = response.json()["firstId"]
        lastId = response.json()["lastId"]
        count = response.json()["count"]
    else:
        print(f"Erro: {response.status_code} - {response.text}")

In [ ]:
url = "https://api.binance.com/api/v3/openOrders"
params = {"symbol": "BTCUSDT"}

response = requests.get(url, params=params)
response.json()


## TEST

In [1]:
def detect_pattern(data, pattern_function, pattern_name: str, dates):
    """
    General method to detect a specific candlestick pattern.
    """

    detection = pattern_function(data['Open'], data['High'], data['Low'], data['Close'])
    detected_indices = np.nonzero(detection)[0]

    results = []
    three_months_ago = datetime.now() - timedelta(days=90)

    for i in detected_indices:
        date = dates[i]

        # Ignorar datas antigas irrelevantes
        if datetime.strptime(date, "%Y-%m-%d %H:%M") < three_months_ago:
            continue  

        signal = int(detection[i])

        # Determinar stop-loss com base no tipo de padrão
        if pattern_name in [
            "doji", "dragonfly_doji", "gravestone_doji", "engulfing",
            "morning_star", "evening_star", "marubozu", "harami",
            "harami_cross", "kicking", "kicking_by_length", "tasuki_gap",
            "gap_side_by_side_white", "counter_attack", "piercing",
            "dark_cloud_cover", "tri_star", "spinning_top"
        ]:
            stoploss = round(data['Low'][i], 5) if signal > 0 else round(data['High'][i], 5)
        elif pattern_name in [
            "morning_doji_star", "hammer", "inverted_hammer",
            "thrusting", "matching_low", "three_white_soldiers",
            "three_outside", "three_stars_in_south"
        ]:
            stoploss = round(data['Low'][i], 5)
        elif pattern_name in [
            "evening_doji_star", "hanging_man", "shooting_star",
            "on_neck", "in_neck", "three_black_crows",
            "three_inside", "advance_block", "stalled_pattern"
        ]:
            stoploss = round(data['High'][i], 5)
        else:
            stoploss = None

        # Verificação se o Stoploss foi atingido
        hit_stoploss = "N/A"

        if stoploss is not None:
            future_close_prices = data['Close'][i + 1:]

            if signal == -100:
                hit_stoploss = "Hit Stoploss (Above)" if any(close > stoploss for close in future_close_prices) else "No Hit"

            elif signal == 100:
                hit_stoploss = "Hit Stoploss (Below)" if any(close < stoploss for close in future_close_prices) else "No Hit"

        # Adicionar ao resultado apenas se houver valores válidos
        if stoploss:
            results.append({
                'Signal': signal,
                'Stoploss': stoploss,
                'Date': date,
                'Result': hit_stoploss
            })

    return results


In [2]:
def doji(self, data, dates): return self.detect_pattern(data, talib.CDLDOJI, "doji", dates)
def dragonfly_doji(self, data, dates): return self.detect_pattern(data, talib.CDLDRAGONFLYDOJI, "dragonfly_doji", dates)
def gravestone_doji(self, data, dates): return self.detect_pattern(data, talib.CDLGRAVESTONEDOJI, "gravestone_doji", dates)
def engulfing(self, data, dates): return self.detect_pattern(data, talib.CDLENGULFING, "engulfing", dates)
def morning_star(self, data, dates): return self.detect_pattern(data, talib.CDLMORNINGSTAR, "morning_star", dates)
def evening_star(self, data, dates): return self.detect_pattern(data, talib.CDLEVENINGSTAR, "evening_star", dates)
def morning_doji_star(self, data, dates): return self.detect_pattern(data, talib.CDLMORNINGDOJISTAR, "morning_doji_star", dates)
def evening_doji_star(self, data, dates): return self.detect_pattern(data, talib.CDLEVENINGDOJISTAR, "evening_doji_star", dates)
def hammer(self, data, dates): return self.detect_pattern(data, talib.CDLHAMMER, "hammer", dates)
def inverted_hammer(self, data, dates): return self.detect_pattern(data, talib.CDLINVERTEDHAMMER, "inverted_hammer", dates)
def hanging_man(self, data, dates): return self.detect_pattern(data, talib.CDLHANGINGMAN, "hanging_man", dates)
def shooting_star(self, data, dates): return self.detect_pattern(data, talib.CDLSHOOTINGSTAR, "shooting_star", dates)
def marubozu(self, data, dates): return self.detect_pattern(data, talib.CDLMARUBOZU, "marubozu", dates)
def harami(self, data, dates): return self.detect_pattern(data, talib.CDLHARAMI, "harami", dates)
def harami_cross(self, data, dates): return self.detect_pattern(data, talib.CDLHARAMICROSS, "harami_cross", dates)
def spinning_top(self, data, dates): return self.detect_pattern(data, talib.CDLSPINNINGTOP, "spinning_top", dates)
def kicking(self, data, dates): return self.detect_pattern(data, talib.CDLKICKING, "kicking", dates)
def kicking_by_length(self, data, dates): return self.detect_pattern(data, talib.CDLKICKINGBYLENGTH, "kicking_by_length", dates)
def tasuki_gap(self, data, dates): return self.detect_pattern(data, talib.CDLTASUKIGAP, "tasuki_gap", dates)
def gap_side_by_side_white(self, data, dates): return self.detect_pattern(data, talib.CDLGAPSIDESIDEWHITE, "gap_side_by_side_white", dates)
def counter_attack(self, data, dates): return self.detect_pattern(data, talib.CDLCOUNTERATTACK, "counter_attack", dates)
def piercing(self, data, dates): return self.detect_pattern(data, talib.CDLPIERCING, "piercing", dates)
def dark_cloud_cover(self, data, dates): return self.detect_pattern(data, talib.CDLDARKCLOUDCOVER, "dark_cloud_cover", dates)
def tri_star(self, data, dates): return self.detect_pattern(data, talib.CDLTRISTAR, "tri_star", dates)
def on_neck(self, data, dates): return self.detect_pattern(data, talib.CDLONNECK, "on_neck", dates)
def in_neck(self, data, dates): return self.detect_pattern(data, talib.CDLINNECK, "in_neck", dates)
def thrusting(self, data, dates): return self.detect_pattern(data, talib.CDLTHRUSTING, "thrusting", dates)
def matching_low(self, data, dates): return self.detect_pattern(data, talib.CDLMATCHINGLOW, "matching_low", dates)
def three_black_crows(self, data, dates): return self.detect_pattern(data, talib.CDL3BLACKCROWS, "three_black_crows", dates)
def three_white_soldiers(self, data, dates): return self.detect_pattern(data, talib.CDL3WHITESOLDIERS, "three_white_soldiers", dates)
def three_inside(self, data, dates): return self.detect_pattern(data, talib.CDL3INSIDE, "three_inside", dates)
def three_outside(self, data, dates): return self.detect_pattern(data, talib.CDL3OUTSIDE, "three_outside", dates)
def three_stars_in_south(self, data, dates): return self.detect_pattern(data, talib.CDL3STARSINSOUTH, "three_stars_in_south", dates)
def advance_block(self, data, dates): return self.detect_pattern(data, talib.CDLADVANCEBLOCK, "advance_block", dates)
def stalled_pattern(self, data, dates): return self.detect_pattern(data, talib.CDLSTALLEDPATTERN, "stalled_pattern", dates)